In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from rdflib import ConjunctiveGraph, URIRef, RDF, RDFS, OWL, Namespace

In [7]:
# loop over all uris in embeddings search for inverse consistsOf
# group of all PL1, PL2, and PL3 entities respectively with given type assignment in trainings data
# same for test data (what about classes never seen in trainings data?)
g_train = ConjunctiveGraph()
g_valid = ConjunctiveGraph()
g_test = ConjunctiveGraph()

base_path = '/home/nether-nova/Documents/Amberg Events/routing_data/'
train_path = base_path + 'train.xml'
valid_path = base_path + 'valid.xml'
test_path = base_path + 'test.xml'

g_train.load(train_path)
g_valid.load(valid_path)
g_test.load(test_path)

CPS_NS = Namespace('http://siemens.com/knowledge_graph/cyber_physical_systems/industrial_cps#')
CONSISTS_OF = CPS_NS['consistsOf']

In [8]:
g_all = ConjunctiveGraph()
g_all.load(base_path + '/Ontology/PPR_individuals.rdf')

def pl_entities(g_all):
    PL1_list = []
    PL2_list = []
    PL3_list = []
    for s,p,o in g_all.triples((None, CONSISTS_OF, None)):
        test_PL1 = list(g_all.triples((None, CONSISTS_OF, s)))
        if len(test_PL1) == 0:
            PL1_list.append((s, get_type(s, g_all)))
        test_PL3 = list(g_all.triples((o, CONSISTS_OF, None)))
        if len(test_PL3) == 0:
            PL3_list.append((o, get_type(o, g_all)))
            PL2_list.append((s, get_type(s, g_all)))
    return PL1_list, PL2_list, PL3_list

def get_type(ent, g):
    types = list(g.objects(ent, RDF.type))
    types = [t for t in types if t != OWL.NamedIndividual]
    return types

pl1_data, pl2_data, pl3_data = pl_entities(g_all)

In [9]:
embeddings = pd.read_csv(base_path + 'Embeddings/entity_embeddings1.csv')

In [10]:
print embeddings

        id        x0        x1        x2        x3        x4        x5  \
0        0  0.272604  0.202814 -0.302922 -0.210899 -0.194981 -0.253227   
1        1  0.175648  0.206869 -0.187791 -0.325227 -0.012099 -0.101467   
2        2  0.219376  0.042716 -0.108244 -0.087803 -0.228621 -0.253545   
3        3  0.076211  0.294644  0.194880 -0.393098 -0.348158 -0.015136   
4        4 -0.226190  0.168551  0.107395 -0.078592  0.094782  0.306853   
5        5  0.233079 -0.256372 -0.164244 -0.083000 -0.436164 -0.179953   
6        6  0.016690  0.222230 -0.271845  0.340455  0.106969 -0.376928   
7        7 -0.057696 -0.284100 -0.143568  0.125938  0.066892 -0.322341   
8        8 -0.098164 -0.012804  0.358927  0.338752  0.234876  0.137114   
9        9  0.203221  0.314733 -0.329370  0.214850 -0.228722 -0.245545   
10      10  0.012533  0.136714 -0.280684 -0.139752  0.291491  0.106011   
11      11  0.161232 -0.249872 -0.250863 -0.245497 -0.163318 -0.442343   
12      12  0.191922  0.029212 -0.2822